# Hybrid
* Movie title and user as input
* Use content based to compute 25 most similar movies
* Compute the predicted ratings using a collaborative model
Return top 10

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 100000 ratings 9000movies 700 users
ratings = pd.read_csv('../data/ratings_small.csv')
ratings.head(50)


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [3]:
ratings.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
userId,100004.0,3.470113e+02,1.951638e+02,1.0,182.0,3.670000e+02,5.200000e+02,6.710000e+02
movieId,100004.0,1.254866e+04,2.636920e+04,1.0,1028.0,2.406500e+03,5.418000e+03,1.639490e+05
rating,100004.0,3.543608e+00,1.058064e+00,0.5,3.0,4.000000e+00,4.000000e+00,5.000000e+00
timestamp,100004.0,1.129639e+09,1.916858e+08,789652009.0,965847824.0,1.110422e+09,1.296192e+09,1.476641e+09


In [4]:
r_matrix = ratings.pivot_table(values='rating',index ='movieId', columns ='userId')
r_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
movieId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,4.0,NaN,...,NaN,4.0,3.5,NaN,NaN,NaN,NaN,NaN,4.0,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
r_matrix.tail()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
movieId,,,,,,,,,,,,,,,,,,,,,
161944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
r_matrix.shape

(9066, 671)

In [7]:
r_matrix_dummy =r_matrix.copy().fillna(0)

In [8]:
r_matrix_dummy.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,...,0.0,4.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,5.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
r_matrix_dummy.to_csv('../output/dummy.csv')

In [9]:
r_matrix_dummy.loc[72998]

userId
1      0.0
2      0.0
3      0.0
4      0.0
5      0.0
6      0.0
7      0.0
8      0.0
9      0.0
10     0.0
11     0.0
12     0.0
13     0.0
14     0.0
15     4.0
16     0.0
17     0.0
18     0.0
19     0.0
20     0.0
21     0.0
22     0.0
23     0.0
24     0.0
25     0.0
26     3.5
27     0.0
28     0.0
29     0.0
30     0.0
      ... 
642    0.0
643    0.0
644    0.0
645    0.0
646    0.0
647    0.0
648    0.0
649    0.0
650    0.0
651    0.0
652    0.0
653    0.0
654    0.0
655    4.0
656    0.0
657    0.0
658    0.0
659    0.0
660    4.0
661    0.0
662    0.0
663    0.0
664    4.0
665    0.0
666    0.0
667    0.0
668    0.0
669    0.0
670    0.0
671    0.0
Name: 72998, Length: 671, dtype: float64

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

In [12]:
cosine_sim = pd.DataFrame(cosine_sim, index =r_matrix.index,columns =r_matrix.index)

In [13]:
cosine_sim.tail()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
movieId,,,,,,,,,,,,,,,,,,,,,
161944,0.079755,0.137594,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
162376,0.079755,0.068797,0.0,0.0,0.0,0.111103,0.000000,0.0,0.0,0.076835,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
162542,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
162672,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
163949,0.055829,0.000000,0.0,0.0,0.0,0.061724,0.079399,0.0,0.0,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Bypass Experiment

In [65]:
# Change series to df
df = cosine_sim.loc[162542].to_frame(name=None)
df.head()

,162542
movieId,
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0


In [75]:
df = df.sort_values(by =[162542],ascending = False).head(25)
df

,162542
movieId,
162672,1.000000
162542,1.000000
98000,1.000000
149606,1.000000
108514,1.000000
150548,0.553372
73290,0.489702
86880,0.372572
3404,0.364390


In [83]:
ls = df.index.to_list()


list

# End experiment

In [14]:
movies_list = pd.read_csv('../data/movie_ids.csv')
movies_list.shape

(9219, 3)

In [15]:
movies_list.head()

,title,movieId,id
0,Toy Story,1,862.0
1,Jumanji,2,8844.0
2,Grumpier Old Men,3,15602.0
3,Waiting to Exhale,4,31357.0
4,Father of the Bride Part II,5,11862.0


In [16]:
movienamepicker = movies_list.set_index('movieId')
movienamepicker.tail()

,title,id
movieId,,
161944,The Last Brickmaker in America,159550.0
162542,Rustom,392572.0
162672,Mohenjo Daro,402672.0
163056,Shin Godzilla,315011.0
163949,The Beatles: Eight Days a Week - The Touring Y...,391698.0


In [17]:
movienamepicker.loc[9000]

title    That's Entertainment! III
id                           77448
Name: 9000, dtype: object

In [18]:
#cosine_sim_map =pd.Series(movies_list.index, index=movies_list['title'])
cosine_sim_map = movies_list.set_index('title')
#del cosine_sim_map.index.name

cosine_sim_map.head()


,movieId,id
title,,
Toy Story,1,862.0
Jumanji,2,8844.0
Grumpier Old Men,3,15602.0
Waiting to Exhale,4,31357.0
Father of the Bride Part II,5,11862.0


In [21]:
#Fucnction tester(idx)
idx =cosine_sim_map.loc['Rustom','movieId']
print(idx)

162542.0


In [22]:
#Fucnction tester(idx) for avatar
idx =cosine_sim_map.loc['Rustom','movieId']
print(idx)

162542.0


In [23]:
# Takes title and returns idx
def idx(title):
    idx =cosine_sim_map.loc[title,'movieId']

    return idx
x = idx('Rustom')
print(x)
    

162542.0


In [31]:
#observed error with avatar 72998
# Takes title and returns to 25 sorted sim_scores
def sim_scores(title):
    idx =cosine_sim_map.loc[title,'movieId']
    #works sim_scores = list(enumerate(list(cosine_sim.iloc[int(idx)])))
    sim_scores = list(enumerate(cosine_sim.loc[int(idx)])) # identified error iloc is row number 
    sim_scores = sorted(sim_scores, key =lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:26]
    
    return sim_scores
x = sim_scores('Rustom')
print(x)

    

[(8475, 1.0), (9006, 1.0), (9063, 1.0), (9064, 1.0), (9010, 0.5533715710928597), (7400, 0.48970210687439175), (7782, 0.3725724067886446), (2716, 0.3643900149394215), (8396, 0.29897403710077436), (6463, 0.2357022603955158), (8131, 0.2296046311721166), (6692, 0.21182963643408087), (6685, 0.2043904521230384), (5627, 0.19580898360256077), (1360, 0.18848425873126295), (6424, 0.18199876605254922), (3460, 0.18111789093066347), (5435, 0.16947612543545568), (2893, 0.1398353534043415), (5628, 0.13900960937138318), (1815, 0.12065329648999294), (1453, 0.11877113736973437), (4686, 0.10657008688422943), (521, 0.08986790140419648), (3845, 0.08859949894356295)]


In [49]:
def movieids(title):
    idx =cosine_sim_map.loc[title,'movieId']
    #works sim_scores = list(enumerate(list(cosine_sim.iloc[int(idx)])))
    sim_scores = list(enumerate(cosine_sim.loc[int(idx)]))
    sim_scores = sorted(sim_scores, key =lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:26]
    movieids = [i[0] for i in sim_scores]
    return movieids
y = movieids('Rustom')
print(y)
type(y)

[8475, 9006, 9063, 9064, 9010, 7400, 7782, 2716, 8396, 6463, 8131, 6692, 6685, 5627, 1360, 6424, 3460, 5435, 2893, 5628, 1815, 1453, 4686, 521, 3845]


list

In [50]:
# Takes title and returns to 25 sorted movie names
def movienames(title):
    idx =cosine_sim_map.loc[title,'movieId']
    #works sim_scores = list(enumerate(list(cosine_sim.iloc[int(idx)])))
    sim_scores = list(enumerate(cosine_sim.loc[int(idx)]))
    sim_scores = sorted(sim_scores, key =lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:26]
    movieids = [i[0] for i in sim_scores]
    movienames = movienamepicker.iloc[movieids][['title']]   
    
    
    return movienames
x = movienames('Rustom')
print(x)
type(x)
    

                                        title
movieId                                      
105211                            Enough Said
135569                       Star Trek Beyond
140749                           29th and Gay
140751                          Almost Normal
136018                             Black Mass
71379                     Paranormal Activity
83480                     Season of the Witch
3386                                      JFK
103048                    The Kings of Summer
44840                        The Benchwarmers
95165    Dragon Ball Z: The World's Strongest
51077                             Ghost Rider
50842                      The Boss of It All
8860                                 Cellular
1690                      Alien: Resurrection
43910               Neil Young: Heart of Gold
4340                               The Animal
8340                     Escape from Alcatraz
3605                              King Creole
8861                Resident Evil:

pandas.core.frame.DataFrame

In [ ]:
# Function tester
# Takes title, userId and returns to 25 sorted movie names on est
def movienames(userId,title):
    idx =cosine_sim_map.loc[title,'movieId']
    #works sim_scores = list(enumerate(list(cosine_sim.iloc[int(idx)])))
    sim_scores = list(enumerate(cosine_sim.iloc[int(idx)]))
    sim_scores = sorted(sim_scores, key =lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:26]
    movieids = [i[0] for i in sim_scores]
    movienames = movienamepicker.iloc[movieids][['title']]   
    
    movienames = movienames.reset_index()
    
    movienames['est'] = movienames['movieId'].apply(lambda x: svd.predict(userId, x).est)
    movienames =movienames.sort_values('est', ascending=False)
    
    return movienames
y = movienames(1,'Grumpier Old Men')
print(y)
    


# Prep done use SVD

In [85]:
from surprise import SVD, Reader, Dataset, evaluate

reader = Reader()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader) #need evaluation
data.split(n_folds=5)
svd = SVD()
trainset =data.build_full_trainset()
svd.train(trainset)

/home/scrybe/.local/lib/python3.6/site-packages/surprise/prediction_algorithms/algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


In [ ]:
svd.predict(2, 522)

In [ ]:
evaluate(svd, data, measures =['RMSE'])

In [86]:
# Final Function
# Takes title, userId and returns to 25 sorted movie names on est
def reco(userId,title):
    idx =cosine_sim_map.loc[title,'movieId']
    df = cosine_sim.loc[idx].to_frame(name=None)
    df = df.sort_values(by =[int(idx)],ascending = False).head(25)
    movieids = df.index.to_list()       
    movienames = movienamepicker.iloc[movieids][['title']]   
    movienames = movienames.reset_index()
    movienames['est'] = movienames['movieId'].apply(lambda x: svd.predict(userId, x).est)
    movienames =movienames.sort_values('est', ascending=False)
    
    return movienames.head(10)



In [87]:
reco(1, 'Avatar')

IndexError: positional indexers are out-of-bounds